In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [17]:
# PATHS

# get working directory
path_working = Path.cwd()

path_data_binary = str(path_working) + "\\" + "data" + "\\" + "binary" + "\\"

path_data_triple = str(path_working) + "\\" + "data" + "\\" + "triple" + "\\"

path_data_multiclass = str(path_working) + "\\" + "data" + "\\" + "multiclass" + "\\"

In [18]:
def create_dataframe (path):
    # Get the files from the path provided in the OP
    files = Path(path).glob('*.csv')  # .rglob to create iterator for subdirectories
    
    #Find all csv files in directory and concatenate them
    dfs = list()
    names = list()
    for file in files:
        data = pd.read_csv(file)
        # .stem is method for pathlib objects to get the filename without the extension
        # create another column in dataframe for filename
        data['file'] = file.stem
        names.append(file.stem)
        dfs.append(data)
    
    df = pd.concat(dfs, ignore_index=True)
    
    return df


In [19]:
df_raw_binary = create_dataframe(path_data_binary)

In [20]:
#drop last column taht shows csv filename source:
del df_raw_binary[df_raw_binary.columns[-1]]

df_raw_binary

R1-PA1:VH     R1-PM1:V   R1-PA2:VH     R1-PM2:V   R1-PA3:VH  \
0      70.399324  127673.0908  -49.572308  127648.0176 -169.578319   
1      73.688102  130280.7109  -46.300719  130255.6377 -166.278082   
2      73.733939  130305.7842  -46.254883  130280.7109 -166.232245   
3      74.083443  130581.5902  -45.899649  130556.5169 -165.882741   
4      74.553268  131083.0556  -45.424094  131057.9823 -165.424375   
...          ...          ...         ...          ...         ...   
78372 -58.934439  130706.9565 -178.900342  130681.8832   61.094490   
78373 -59.020382  131057.9823 -178.992015  131032.9090   61.002816   
78374 -59.031842  131083.0556 -179.003474  131032.9090   60.997087   
78375 -59.077678  131108.1288 -179.049311  131057.9823   60.951250   
78376 -59.203729  131158.2754 -179.169632  131108.1288   60.825200   

          R1-PM3:V  R1-PA4:IH   R1-PM4:I   R1-PA5:IH   R1-PM5:I  ...  \
0      127723.2374  65.689611  605.91099  -57.003571  626.78553  ...   
1      130355.9307  71.831719  483.59351  -50.947407  500.98896  ...   
2      130381.0040  71.808800  483.59351  -50.913030  500.98896  ...   
3      130656.8100  72.152575  482.86107  -50.437475  499.15786  ...   
4      131158.2754  72.118198  484.50906  -50.013486  497.69298  ...   
...            ...        ...        ...         ...        ...  ...   
78372  130782.1763 -60.149109  444.59108  178.808669  449.90127  ...   
78373  131108.1288 -61.507019  451.73237  177.857559  454.66213  ...   
78374  131133.2021 -61.535667  451.91548  177.857559  454.66213  ...   
78375  131158.2754 -61.839335  453.38036  177.725778  455.02835  ...   
78376  131208.4219 -62.446670  455.39457  177.336167  456.12701  ...   

       control_panel_log4  relay1_log  relay2_log  relay3_log  relay4_log  \
0                       0           0           0           0           0   
1                       0           0           0           0           0   
2                       0           0           0           0           0   
3                       0           0           0           0           0   
4                       0           0           0           0           0   
...                   ...         ...         ...         ...         ...   
78372                   0           0           0           0           0   
78373                   0           0           0           0           0   
78374                   0           0           0           0           0   
78375                   0           0           0           0           0   
78376                   0           0           0           0           0   

       snort_log1  snort_log2  snort_log3  snort_log4   marker  
0               0           0           0           0  Natural  
1               0           0           0           0  Natural  
2               0           0           0           0  Natural  
3               0           0           0           0  Natural  
4               0           0           0           0  Natural  
...           ...         ...         ...         ...      ...  
78372           0           0           0           0  Natural  
78373           0           0           0           0  Natural  
78374           0           0           0           0  Natural  
78375           0           0           0           0  Natural  
78376           0           0           0           0  Natural  

[78377 rows x 129 columns]

In [21]:
print(df_raw_binary.shape)

df_raw_binary['marker'].value_counts()

(78377, 129)


Attack     55663
Natural    22714
Name: marker, dtype: int64

### ---> unbalanced classes

In [22]:
# .copy() is very important otherwise its just a pointer
df = df_raw_binary.copy()

In [23]:
def marker_to_numeric_binary(x):
        if x=='Attack': return 0
        if x=='Natural': return 1
        
LABEL_MAP = {0 : "Attack", 1: "Natural"}

LABELNAMES = [LABEL_MAP[key] for key in LABEL_MAP]

df['marker'] = df['marker'].apply(marker_to_numeric_binary)
df

R1-PA1:VH     R1-PM1:V   R1-PA2:VH     R1-PM2:V   R1-PA3:VH  \
0      70.399324  127673.0908  -49.572308  127648.0176 -169.578319   
1      73.688102  130280.7109  -46.300719  130255.6377 -166.278082   
2      73.733939  130305.7842  -46.254883  130280.7109 -166.232245   
3      74.083443  130581.5902  -45.899649  130556.5169 -165.882741   
4      74.553268  131083.0556  -45.424094  131057.9823 -165.424375   
...          ...          ...         ...          ...         ...   
78372 -58.934439  130706.9565 -178.900342  130681.8832   61.094490   
78373 -59.020382  131057.9823 -178.992015  131032.9090   61.002816   
78374 -59.031842  131083.0556 -179.003474  131032.9090   60.997087   
78375 -59.077678  131108.1288 -179.049311  131057.9823   60.951250   
78376 -59.203729  131158.2754 -179.169632  131108.1288   60.825200   

          R1-PM3:V  R1-PA4:IH   R1-PM4:I   R1-PA5:IH   R1-PM5:I  ...  \
0      127723.2374  65.689611  605.91099  -57.003571  626.78553  ...   
1      130355.9307  71.831719  483.59351  -50.947407  500.98896  ...   
2      130381.0040  71.808800  483.59351  -50.913030  500.98896  ...   
3      130656.8100  72.152575  482.86107  -50.437475  499.15786  ...   
4      131158.2754  72.118198  484.50906  -50.013486  497.69298  ...   
...            ...        ...        ...         ...        ...  ...   
78372  130782.1763 -60.149109  444.59108  178.808669  449.90127  ...   
78373  131108.1288 -61.507019  451.73237  177.857559  454.66213  ...   
78374  131133.2021 -61.535667  451.91548  177.857559  454.66213  ...   
78375  131158.2754 -61.839335  453.38036  177.725778  455.02835  ...   
78376  131208.4219 -62.446670  455.39457  177.336167  456.12701  ...   

       control_panel_log4  relay1_log  relay2_log  relay3_log  relay4_log  \
0                       0           0           0           0           0   
1                       0           0           0           0           0   
2                       0           0           0           0           0   
3                       0           0           0           0           0   
4                       0           0           0           0           0   
...                   ...         ...         ...         ...         ...   
78372                   0           0           0           0           0   
78373                   0           0           0           0           0   
78374                   0           0           0           0           0   
78375                   0           0           0           0           0   
78376                   0           0           0           0           0   

       snort_log1  snort_log2  snort_log3  snort_log4  marker  
0               0           0           0           0       1  
1               0           0           0           0       1  
2               0           0           0           0       1  
3               0           0           0           0       1  
4               0           0           0           0       1  
...           ...         ...         ...         ...     ...  
78372           0           0           0           0       1  
78373           0           0           0           0       1  
78374           0           0           0           0       1  
78375           0           0           0           0       1  
78376           0           0           0           0       1  

[78377 rows x 129 columns]

In [24]:
# Check for infinite float entries
# "np.finfo(np.float64).max" returns the maximum finite value that can be represented by a 64-bit floating point number
# "df.values >= np.finfo(np.float64).max" returns a bollean array where "np.finfo(np.float64).max" is checked
# "np.where(df.values >= np.finfo(np.float64).max)" returns a array of indices where "df.values >= np.finfo(np.float64).max" is true

np.where(df.values >= np.finfo(np.float64).max)

(array([  738,   739,   753, ..., 78352, 78353, 78353], dtype=int64),
 array([26, 26, 26, ..., 55, 26, 55], dtype=int64))

### ----> we have infinite float entries

In [25]:
# Check for NaN values
np.isnan(df.values.any())

False

### ----> we have no NaN values in the dataframe

In [26]:
# Replace infinite updated data with nan

# "[np.inf, -np.inf]" specifies the values to be replaced
# "np.nan" specifies the value to replace the infinite values with.
# "inplace=True" modifies the Dataframe in place, so that the original df is updated the new values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN
df.dropna(inplace=True)

print(df.shape)

df['marker'].value_counts()

(72073, 129)


0    51445
1    20628
Name: marker, dtype: int64

### ----> Due to cleaning up NaNs and infinites we lost about 6000 datapoints, which is not very harmful due to the fact that there are still 72000 rows

In [27]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

#This only copies a Series Object, no pandas Dataframe!
df_marker = df["marker"].copy()
#Converting to Pandas Dataframe
df_marker = pd.DataFrame(df_marker)

#Delete Marker column because "marker" should not be normalized
del df[df.columns[-1]]

#Initialize SCaler
scaler = StandardScaler()
#Scale data
data_normalized = scaler.fit_transform(df)
#insert normalized data in former pandas frame
df_normalized_nomark = pd.DataFrame(data_normalized, columns=df.columns)

#concat "marker" column with normalized rest dataframe, index of marker frame is overwritten
df_normalized = pd.concat([df_normalized_nomark, df_marker.set_index(df_normalized_nomark.index)], axis=1)

#print(df_normalized_nomark)

#print(isinstance(df, pd.DataFrame))

#print(isinstance(df_marker, pd.DataFrame))

#print(df_normalized)


In [28]:
df_normalized.iloc[:,128].unique()

array([1, 0], dtype=int64)

In [29]:
y = df_normalized['marker']

X = df_normalized.drop(['marker'],axis =1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((43243, 128), (43243,), (14415, 128), (14415,), (14415, 128), (14415,))

In [31]:
# Convert to numpy array

X_train_np = X_train.to_numpy() 

y_train_np = y_train.to_numpy() 

X_val_np = X_val.to_numpy() 

y_val_np = y_val.to_numpy() 

X_test_np = X_test.to_numpy() 

y_test_np = y_test.to_numpy() 

In [32]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

In [33]:
# Convert to torch tensors

X_train_tensor = T.tensor(X_train_np, dtype=T.float32)

y_train_tensor = T.tensor(y_train_np, dtype=T.long)

X_val_tensor = T.tensor(X_val_np, dtype=T.float32)

y_val_tensor = T.tensor(y_val_np, dtype=T.long)

X_test_tensor = T.tensor(X_test_np, dtype=T.float32)

y_test_tensor = T.tensor(y_test_np, dtype=T.long)

In [34]:
device = T.device("cpu")

In [35]:
class MyDataset(Dataset):
    def __init__(self, feature_tensor, target_tensor):
        self.feature_tensor = feature_tensor
        self.target_tensor = target_tensor
        
    def __len__(self):
        return len(self.feature_tensor)
    
    def __getitem__(self, idx):
        features = self.feature_tensor[idx]
        target = self.target_tensor[idx]
        
        return features, target

In [36]:
class ThreeClassesDataset(T.utils.data.Dataset):
    
    def __init__(self, src_file):
        all_xy_np = np.array(src_file.values, dtype=np.float64)
        
        tmp_x_np = all_xy_np[:,0:128]
        
        tmp_y_np = all_xy_np[:,128] 
        
        self.x_data = T.from_numpy(tmp_x_np).to(device)
        
        
        self.y_data = T.from_numpy(tmp_y_np).to(device)
        
    def __len__(self):
        return(self.x_data)
    
    def __getitem__(self,idx):
        preds = self.x_data[idx]
        targets = self.y_data[idx]
        return preds, targets # as a tuple
    

In [37]:
class Net(T.nn.Module):  # MULTICLASS!!!
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = T.nn.Linear(128,10)
        self.hid2 = T.nn.Linear(10,10)
        self.output = T.nn.Linear(10,3) # Number of Classes to predict!
        
        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.hid2.weight)
        T.nn.init.zeros_(self.hid2.bias)
        T.nn.init.xavier_uniform_(self.output.weight)
        T.nn.init.zeros_(self.output.bias)
        
    def forward(self, x):
        z = T.tanh(self.hid1(x))
        z = T.tanh(self.hid2(z))
        z = T.log_softmax(self.output(z), dim=1) # NLLLoss() 
        return z        

In [38]:
class Net(nn.Module): # BINARY !!!
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(128, 10)
        self.fc2 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = T.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

In [39]:
class Net(T.nn.Module):  # BINARY !!!
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = T.nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = T.nn.BatchNorm1d(num_features=32)
        self.pool1 = T.nn.MaxPool1d(kernel_size=2)
        self.conv2 = T.nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = T.nn.BatchNorm1d(num_features=64)
        self.pool2 = T.nn.MaxPool1d(kernel_size=2)
        self.fc1 = T.nn.Linear(in_features=64*32, out_features=128)
        self.dropout = T.nn.Dropout(p=0.5)
        self.fc2 = T.nn.Linear(in_features=128, out_features=2) # Binary classification
        
    def forward(self, x):
        x = x.unsqueeze(1)  # add channel dimension
        x = self.pool1(T.relu(self.bn1(self.conv1(x))))
        x = self.pool2(T.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64*32)
        x = T.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [40]:
train_ds = MyDataset(X_train_tensor, y_train_tensor)

val_ds = MyDataset(X_val_tensor, y_val_tensor)

test_ds = MyDataset(X_test_tensor, y_test_tensor)



bat_size = 1000

train_loader = DataLoader(train_ds, batch_size = bat_size, shuffle=True, drop_last = True)

val_loader = DataLoader(val_ds, batch_size = bat_size, shuffle=True, drop_last = True)

test_loader = DataLoader(test_ds, batch_size = bat_size, shuffle=True, drop_last = True)

#shuffle = True is extremely important because training the networks fails throtugh oscillationg biases and weights
#drop_last = True is good practice because the batch sizes should be evenly divided




In [12]:
# create Network
print("Creating 128-(10-10)-3 neural network ")
net = Net().to(device)
net.train()

Creating 128-(10-10)-3 neural network 


Net(
  (conv1): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [52]:
 # 3. train model
max_epochs = 1000
ep_log_interval = 100
lrn_rate = 0.01
val_interval = 100

loss_func = T.nn.CrossEntropyLoss()  # T.nn.NLLLoss() for multiple classification, T.nn.CrossEntropyLoss() for binary classification
optimizer = T.optim.SGD(net.parameters(),lr=lrn_rate)

print("bat_size = %3d " % bat_size)
print("loss = " + str(loss_func))
print("optimizer = SGD")
print("max_epochs = %3d " % max_epochs)
print("lrn_rate = %0.3f " % lrn_rate)
print("Starting training ")
for epoch in range(0, max_epochs):
    # T.manual_seed(epoch+1)  # reproducibility
    epoch_loss = 0  # for one full epoch

    for (batch_idx, batch) in enumerate(train_loader):
        X = batch[0]  # inputs
        Y = batch[1]  # correct class/label/politics

        optimizer.zero_grad()
        oupt = net(X)
        loss_val = loss_func(oupt, Y)  # a tensor
        epoch_loss += loss_val.item()  # accumulate
        loss_val.backward()
        optimizer.step()

    if epoch % ep_log_interval == 0:
        print("epoch = %5d  |  loss = %10.4f" % \
        (epoch, epoch_loss))
        
    # Add validation loop
    if epoch % val_interval == 0:
        net.eval() # Set model to evaluation mode
        val_loss = 0
        with T.no_grad(): # Disable gradient calculation to save memory
            for (batch_idx, batch) in enumerate(val_loader):
                X_val = batch[0]
                Y_val = batch[1]
                oupt_val = net(X_val)
                val_loss += loss_func(oupt_val, Y_val).item()

        print("Validation loss = %10.4f" % val_loss)

        net.train() # Set model back to training mode
        
print("Training done ")

bat_size = 1000 
loss = CrossEntropyLoss()
optimizer = SGD
max_epochs = 1000 
lrn_rate = 0.010 
Starting training 
epoch =     0  |  loss =    26.2514
Validation loss =     8.3725
epoch =   100  |  loss =    22.9403
Validation loss =     7.5859
epoch =   200  |  loss =    21.2321
Validation loss =     7.0753
epoch =   300  |  loss =    20.3502
Validation loss =     7.0796
epoch =   400  |  loss =    19.6388
Validation loss =     6.8366
epoch =   500  |  loss =    18.9497
Validation loss =     6.7086
epoch =   600  |  loss =    18.4221
Validation loss =     6.6708
epoch =   700  |  loss =    18.1895
Validation loss =     6.5602
epoch =   800  |  loss =    17.6880
Validation loss =     7.0293
epoch =   900  |  loss =    17.3225
Validation loss =     6.5345
Training done 


In [53]:
def accuracy(model, ds):
    # assumes model.eval()
    # item-by-item version
    n_correct = 0; n_wrong = 0
    for i in range(len(ds)):
        X = ds[i][0].reshape(1,-1)  # make it a batch
        Y = ds[i][1].reshape(1)  # 0 1 or 2, 1D
        with T.no_grad():
            oupt = model(X)  # logits form

        big_idx = T.argmax(oupt)  # 0 or 1 or 2
        if big_idx == Y:
            n_correct += 1
        else:
            n_wrong += 1

    acc = (n_correct * 1.0) / (n_correct + n_wrong)
    return acc

In [54]:
# 4. evaluate model accuracy
print("Computing model accuracy ")
net.eval()
acc_train = accuracy(net, train_ds)  # item-by-item
print("Accuracy on training data = %0.4f" % acc_train)
acc_test = accuracy(net, test_ds) 
print("Accuracy on test data = %0.4f" % acc_test)

Computing model accuracy 
Accuracy on training data = 0.8253
Accuracy on test data = 0.7842


In [13]:
# Define the file path to the pre-trained model
model_path = 'binary_1DCNN_model'

# Load the model state dictionary
state_dict = T.load(model_path)

# Create a new model instance
model = Net()

# Load the state dictionary into the model
model.load_state_dict(state_dict)

<All keys matched successfully>

In [41]:
def evaluate_model(model, test_loader, labelmap, device='cpu', binarylabels=False):
    """
    Evaluate a multi-class or binary classification CNN model using various metrics.

    Args:
        model (PyTorch model): The model to evaluate.
        test_loader (PyTorch data loader): The test data loader.
        labelmap (dict): The encoding of the names of the labels that are passed to report and matrix.
        device (str): The device on which to run the evaluation.
        binarylabels (bool): If True, the model was built for binary classification.

    Returns:
        sklearn confusion matrix, sklearn report, metrics (if binarylabels is True).
    """

    # set model to evaluation mode
    model.eval()

    # get class names
    labelnames = [labelmap[key] for key in labelmap]

    # initialize lists to store predicted and true labels
    y_pred_list = []
    y_true_list = []

    with T.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = T.max(outputs.data, 1)
            y_true_list += labels.cpu().numpy().tolist()
            y_pred_list += predicted.cpu().numpy().tolist()

    # map encoded labels to original labels
    y_true = [labelmap[i] for i in y_true_list]
    y_pred = [labelmap[i] for i in y_pred_list]

    # compute the confusion matrix and print it
    confusionmatrix = confusion_matrix(y_true, y_pred, labels=labelnames)
    print("Confusion matrix:")
    print(confusionmatrix)

    # compute the classification report and print it
    report = classification_report(y_true, y_pred, target_names=labelnames)
    print("Classification report:")
    print(report)

    if binarylabels:
        # calculate metrics out of confusion matrix to compare
        tp, fn, fp, tn = confusionmatrix.ravel()  # first row and first column correspond to positive class

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        specificity = tn / (tn + fp)

        metrics = [precision, recall, specificity]

    else:
        metrics = None

    return confusionmatrix, report, metrics

In [42]:
cm, rp, mets = evaluate_model(net, test_loader, LABEL_MAP, device='cpu', binarylabels = True )

Confusion matrix:
[[  38 9977]
 [  15 3970]]
Classification report:
              precision    recall  f1-score   support

      Attack       0.72      0.00      0.01     10015
     Natural       0.28      1.00      0.44      3985

    accuracy                           0.29     14000
   macro avg       0.50      0.50      0.23     14000
weighted avg       0.59      0.29      0.13     14000



In [67]:
print(mets)

[0.8027900528550386, 0.9258519036674328, 0.4300025043826697]


In [1]:
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELNAMES, yticklabels=LABELNAMES)

NameError: name 'sns' is not defined

In [70]:
# save the model

path_save_model = str(path_working) + "\\" + "binary_1DCNN_model"

T.save(net.state_dict(), path_save_model)